# Setup

In [152]:
import importlib
import brikasutils as bu
importlib.reload(bu)
import fb_msg_reader as fb
importlib.reload(fb)
import shared_utils as utils
importlib.reload(utils)
import survey
importlib.reload(survey)
import persona
importlib.reload(persona)
import embed
importlib.reload(embed)

import ollama
import numpy as np
from numpy.linalg import norm
import pandas as pd
import os
import re
import time
import json
from typing import List
from openai import OpenAI

In [153]:
# Prompt builder
def userMsg(*args) -> dict:
    return {"role": "user", "content": "\n".join(args)}
def assistantMsg(*args) -> dict:
    return {"role": "assistant", "content": "\n".join(args)}
def systemMsg(*args) -> dict:
    return {"role": "system", "content": "\n".join(args)}

## Load data

In [154]:
elias = persona.PersonaEncoder()
# ==== pre-Cleaning ====
# texts_with_elias = [
#     "selected-data/elias/message_1.json",
#     "selected-data/petyo/message_2.json",
# ]
# ==== FB messages ====
elias.parse_fb_messages(["data-raw/1_airidas.json"], "airidas")
elias.parse_fb_messages(["data-raw/2_christian.json"], "christian")
elias.parse_fb_messages(["data-raw/1_nikolay.json"], "nikolay")
elias.parse_fb_messages(["data-raw/2_mathis.json"], "mathis")
elias.parse_fb_messages(["data-raw/2_jacob.json"], "jacob")
elias.parse_fb_messages(["data-raw/2_chris.json"], "chris")
elias.parse_fb_messages(["data-raw/3_aziz.json"], "aziz")
elias.parse_fb_messages(["data-raw/3_daniela.json"], "daniela")
elias.parse_fb_messages(["data-raw/3_mihi.json"], "mihi")
elias.parse_fb_messages(["data-raw/3_viktoria.json"], "viktoria")
elias.parse_fb_messages(["data-raw/4_diba.json"], "diba")
elias.parse_fb_messages(["data-raw/6_filip.json"], "filip")
#elias.parse_wa_messages(texts_with_rebecca, "rebecca")

# Regex cleaning
elias.filter_chats_empty()
elias.filter_chats_regex(utils.BLACKLIST_CHAT_REGEX_FILTERS)

# Extra data processiing
# Compress names
for nameid, chat in elias.chats.items():
    for msg in chat:  
        msg.sender = "Persona" if msg.sender == "Elias Salvador Smidt Torjani"  else "Friend"

# Start all chats from 2/3rds
#for name, chat in elias.chats.items():
#    elias.chats[name] = chat[int(len(chat)/3 * 2):]
# Select the final modules
elias.select_chat_limited_by_tokens("airidas", 10000)
elias.select_chat_limited_by_tokens("christian", 10000)
elias.select_chat_full("nikolay")
elias.select_chat_full("mathis") 
elias.select_chat_full("daniela")
elias.select_chat_full("diba")
elias.select_chat_full("aziz")
elias.select_chat_full("jacob")  
elias.select_chat_full("chris")
elias.select_chat_full("filip")
elias.select_chat_full("mihi")
elias.select_chat_full("viktoria")

bu.quickTXT(elias.output(), filename=f"data/big_module")
#bu.quickTXT(persona_text, filename=f"data/_{bu.get_timestamp()}")

Read 1916 msgs from 1 file(s). Failed reading 0 msgs --> 20210913-20240306
Messages saved to self.fb_messages['airidas']
Read 618 msgs from 1 file(s). Failed reading 0 msgs --> 20210910-20240303
Messages saved to self.fb_messages['christian']
Read 297 msgs from 1 file(s). Failed reading 0 msgs --> 20180725-20240101
Messages saved to self.fb_messages['nikolay']
Read 144 msgs from 1 file(s). Failed reading 0 msgs --> 20210328-20211230
Messages saved to self.fb_messages['mathis']
Read 104 msgs from 1 file(s). Failed reading 0 msgs --> 20210825-20240305
Messages saved to self.fb_messages['jacob']
Read 159 msgs from 1 file(s). Failed reading 0 msgs --> 20211012-20230430
Messages saved to self.fb_messages['chris']
Read 161 msgs from 1 file(s). Failed reading 0 msgs --> 20210328-20210606
Messages saved to self.fb_messages['aziz']
Read 350 msgs from 1 file(s). Failed reading 0 msgs --> 20210326-20220411
Messages saved to self.fb_messages['daniela']
Read 105 msgs from 1 file(s). Failed reading 

In [164]:
big_module=elias.output()
utils.count_tokens(big_module)

31830

# Embedding

## Pick parameters of simulation

### Survey

In [184]:
from survey import KanoSurvey, DictatorGameSurvey, PersonalitySurvey
survey = "personality"  # Specify the survey name

# if survey == "kano":
#     csv_file = "surveys/survey_kano.csv",
#     surv = survey.KanoSurvey(csv_file)
# elif survey == "fairness":
#     csv_file = "surveys/survey_fairness.csv"
    # surv = survey.buildFairnessPrompts(csv_file)
if survey == "dictator-game":
    csv_file = "surveys/survey_dictator-game.csv"
    surv = survey.DictatorGameSurvey(csv_file)
elif survey == "personality":
    csv_file = "surveys/survey_personality.csv"
    surv = survey.PersonalitySurvey(csv_file)
else:
    raise ValueError("Invalid survey name")

# Load the CSV file
# df = pd.read_csv(csv_file)

# # Print the loaded survey
# print(df)

# # csv_file = "surveys/survey_kano-model_v1.csv"
# # surv = survey.KanoSurvey(csv_file)
# # csv_file = "surveys/survey_fairness_v1.csv"
# # surv = survey.buildFairnessPrompts(csv_file)
# # csv_file = "surveys/survey_dictator-game_v1.csv"
# # surv = survey.DictatorGameSurvey(csv_file)
# csv_file = "surveys/survey_personality-test_v1.csv"
# surv = survey.PersonalitySurvey(csv_file)
surv.questions[:5]#.head()

AttributeError: 'str' object has no attribute 'PersonalitySurvey'

### Prompt LEGO

In [140]:
# persona_small = "{small module}"
# persona_med = "{med module}"
subject = elias

# Change below accoring to survey above
retrieval_prompt = "openess conciousness extrovert aggreable neuroticism"
# q_retrival_prompt =
survey_prompt = "Determine how much {subject} aggree with the statement. Guestimate how {subject} would answer to the question"

PROMPT = {
    "role": "system", 
    "content": "You are an actor specializing in impersonating non-famouns people. You will be presented to the subject through explicit datapoints of their digital footprint. In addition, you will deduct their implicit personality traits by shadowing chats between the subject and friends. You will be asked to answer questions from the point of view of the persona. The persona you will be impersonating is named Elias. Context:"
    }

## Encoding & retrieve: chat2vec

In [141]:
embed = "nomic-embed-text" #nomic-embed-text = long ctx / mxbai-embed-large = big

# Define parameters
chunk_size = 50     # Number of messages per chunk
overlap = 20         # Number of overlapping messages between consecutive chunks
ctx = 100            # Number of nearby chunks to put in context window

elias.selectedChats["airidas"][:5]

[Message(content='can u take a iphone charger?', sender='Friend', timestamp=1709731251130, seq=2, filename='data-raw/1_airidas.json'),
 Message(content='it has a kids table, you can bring your kaerste too to play with the invisible legos while we do the thesis 😂', sender='Friend', timestamp=1709731112159, seq=3, filename='data-raw/1_airidas.json'),
 Message(content='DH.Ø.0.13', sender='Friend', timestamp=1709730917276, seq=6, filename='data-raw/1_airidas.json'),
 Message(content='I’ll head over there now', sender='Persona', timestamp=1709729828407, seq=7, filename='data-raw/1_airidas.json'),
 Message(content='I meant laundry lmao', sender='Persona', timestamp=1709729809002, seq=8, filename='data-raw/1_airidas.json')]

### Chunking

In [179]:
# different chunk size

# Initialize lists for storing chunks and embeddings
chunks = []
embeddings = []

# Iterate over chats and messages to create chunks
for chat in elias.selectedChats.values():
    messages = list(chat)  # Convert chat iterator to list for easier slicing
    num_messages = len(messages)

    # Create overlapping chunks of messages
    for i in range(0, num_messages - chunk_size + 1, chunk_size - overlap):
        chunk = messages[i:i + chunk_size]  # Extract chunk of messages
        chunk_text = "\n".join(str(msg) for msg in chunk)  # Concatenate messages into a single string
        chunks.append(chunk_text)  # Append chunk to list of chunks

# Generate embeddings for each chunk
for chunk_text in chunks:
    embedding = ollama.embeddings(model=embed, prompt=chunk_text)["embedding"]
    embeddings.append(embedding)

# Perform similarity search and print results
prompt_embedding = ollama.embeddings(model=embed, prompt=retrieval_prompt)["embedding"]
most_similar_chunks = find_most_similar(prompt_embedding, embeddings)[:ctx]

# Display results
print(f"Chunks:{len(chunks)}, embeds:{len(embeddings)}, nearby_chunks:{len(most_similar_chunks)}")
for item in most_similar_chunks:
    print(chunks[item[1]])

TypeError: Object of type module is not JSON serializable

In [38]:
#1.0 of above
paragraphs = []
for chat in elias.selectedChats.values():
    for msg in chat:
        if msg.sender == "Persona":
            paragraphs.append(str(msg))
        
embeddings = [
    ollama.embeddings(model=embed, prompt=chunk)["embedding"] for chunk in paragraphs
]

prompt_embedding = ollama.embeddings(model=embed, prompt=retrieval_prompt)["embedding"]
most_similar_chunks = find_most_similar(prompt_embedding, embeddings)[:5]

print(f"Paragraphs:{len(paragraphs)}, embeds:{len(embeddings)}, nearby_chunks:{len(most_similar_chunks)}")
print("\n".join(paragraphs[item[1]] for item in most_similar_chunks))
#print(f"{most_similar_chunks[0][1]}, {most_similar_chunks[:3]}\n{len(paragraphs)}")

KeyboardInterrupt: 

## Prompt Builder

In [149]:
final_prompts = []

for question in surv.questions:
    p = [
        systemMsg(PROMPT['content']+"\n## chat conversions between subject and friends\n".join(chunks[item[1]] for item in most_similar_chunks)),
        assistantMsg('Understood. I will answer from the point of view of the persona, {subject}, based on what I could the deduct from the text provided above.'),
        userMsg("\n".join([
            f'\n\n**Your answer should only contain the chosen option without further explanation!** Reply to the statement below - how {subject} would reply - with one of these five options: {", ".join(surv.POSSIBLE_ANSWERS)}.',
            question
        ])),
    ]
    final_prompts.append(p)

bu.quickJSON(final_prompts, "data/_prompts.json")
print(f"{len(final_prompts)}")#,{final_prompts[:1]}")

50


# Run simulation

In [150]:
# Run Simulation
##################################
Sim_ID = f"elias_personality"

SETTINGS = {
     "model": "mistral",
     # best wizard and mixtral try mixtral-8x22b wizard in uCloud
}

client = OpenAI(
    base_url = 'http://localhost:11434/v1',
    api_key='ollama', # required, but unused
)

save = f"{SETTINGS['model']}_{Sim_ID}"
##################################

In [151]:
### ==== THE FUNCTIONAL 1!!!! =====

results = []
l = len(final_prompts)
lim = None

for i, (prompt, question) in enumerate(list(zip(final_prompts, surv.questions))):
    if lim != None and i > lim:
        break

    print(f"{i}/{l}...", end="\t") # Print progress
    # Send the Request    
    full_response = client.chat.completions.create(
        model=SETTINGS["model"],
        messages=prompt,
    )
    r = full_response.choices[0].message.content
    # r = full_response['choices'][0]['message']['content']
    # r = full_response

    results.append({'question': question, 'answer': r})

    print(f"{question}: {r}")

# Save results
df = pd.DataFrame(results)
df.to_csv(f"results/{save}_simulation.csv", index=False)
bu.quickJSON(final_prompts, f"results/{save}_prompts.json")
bu.quickJSON(SETTINGS, f"results/{save}_setings.json")

0/50...	I am the life of the party.:  SOMEWHAT AGREE.
1/50...	I don't talk a lot.:  NEUTRAL
2/50...	I feel comfortable around people.:  AGREE.
3/50...	I keep in the background.:  NEUTRAL
4/50...	I start conversations.:  NEUTRAL
5/50...	I have little to say.:  NEUTRAL
6/50...	I talk to a lot of different people at parties.:  NEUTRAL
7/50...	I don't like to draw attention to myself.:  NEUTRAL
8/50...	I don't mind being the center of attention.:  NEUTRAL
9/50...	I am quiet around strangers.:  SOMEWHAT AGREE
10/50...	I get stressed out easily.:  NEUTRAL
11/50...	I am relaxed most of the time.:  NEUTRAL.
12/50...	I worry about things.:  NEUTRAL.
13/50...	I seldom feel blue.:  NEUTRAL
14/50...	I am easily disturbed.:  NEUTRAL
15/50...	I get upset easily.:  NEUTRAL.
16/50...	I change my mood a lot.:  NEUTRAL
17/50...	I have frequent mood swings.:  NEUTRAL
18/50...	I get irritated easily.:  NEUTRAL
19/50...	I often feel blue.:  SOMEWHAT DISAGREE
20/50...	I feel little concern for others.:  NEU

# Analysis

In [116]:
surv.POSSIBLE_ANSWERS[0]

'DISAGREE'

In [123]:
remap_dict = {f"{surv.POSSIBLE_ANSWERS[0]}": 1, f"{surv.POSSIBLE_ANSWERS[1]}": 2, f"{surv.POSSIBLE_ANSWERS[2]}": 3, f"{surv.POSSIBLE_ANSWERS[3]}": 4, f"{surv.POSSIBLE_ANSWERS[4]}": 5}
#remap_dict = {"I EXPECT IT": 5, "I LIKE IT": 4, "I AM NEUTRAL": 3, "I CAN TOLERATE IT": 2, "I DISLIKE IT": 1}

def extract_uppercase_text(text):
    """Extract uppercase text from a string using regex."""
    
    phrases_to_extract = [
        surv.POSSIBLE_ANSWERS[0],
        surv.POSSIBLE_ANSWERS[1],
        surv.POSSIBLE_ANSWERS[2],
        surv.POSSIBLE_ANSWERS[3],
        surv.POSSIBLE_ANSWERS[4],
    #     "I EXPECT IT",
    #     "I LIKE IT",
    #     "I AM NEUTRAL",
    #     "I CAN TOLERATE IT",
    #     "I DISLIKE IT"
    ]
    pattern = r'\b(?:' + '|'.join(re.escape(phrase) for phrase in phrases_to_extract) + r')\b'
    matches = re.findall(pattern, text, flags=re.IGNORECASE) 
    return ' '.join(matches) if matches else ''

In [124]:
#df = pd.read_csv(f'results/{save}_simulation.csv')
df = pd.read_csv('results/wizardlm2_elias_2024-04-16_120635_simulation.csv')
airidas = surv.test_answers["airidas"]
elias = surv.test_answers["elias"]
df.insert(2, "IRL_air", airidas[:len(df)])
df.insert(3, "IRL_eli", elias[:len(df)])
df.head()

,question,answer,IRL_air,IRL_eli
0,I am the life of the party.,🙂,3,3
1,I don't talk a lot.,Neutral,2,2
2,I feel comfortable around people.,👆 Agree (as it reflects agreeing with the sent...,4,4
3,I keep in the background.,👁️‍🗨️ Neutral (This indicates that you neither...,2,4
4,I start conversations.,\nAGREE. You seem to be someone who initiates ...,4,3


In [125]:
df['CLONE_eli'] = df['answer'].apply(extract_uppercase_text)
df['CLONE_eli'] = df['CLONE_eli'].str.upper()
df.head()

,question,answer,IRL_air,IRL_eli,CLONE_eli
0,I am the life of the party.,🙂,3,3,
1,I don't talk a lot.,Neutral,2,2,NEUTRAL
2,I feel comfortable around people.,👆 Agree (as it reflects agreeing with the sent...,4,4,AGREE
3,I keep in the background.,👁️‍🗨️ Neutral (This indicates that you neither...,2,4,NEUTRAL AGREE DISAGREE NEUTRAL
4,I start conversations.,\nAGREE. You seem to be someone who initiates ...,4,3,AGREE


In [126]:
# df = df.drop(columns=['uppercase_text'])
# .str.upper() or .lower()
# df['answer'] = df['answer'].map(remap_dict, na_action='ignore')

df['CLONE_eli'] = df['CLONE_eli'].map(remap_dict)
#df['CLONE_eli'] = df['CLONE_eli'].fillna(0).astype(int)
# df['air'] = df['air'].map(remap_dict)
# df['eli'] = df['eli'].map(remap_dict)
df.head()

,question,answer,IRL_air,IRL_eli,CLONE_eli
0,I am the life of the party.,🙂,3,3,NaN
1,I don't talk a lot.,Neutral,2,2,3.0
2,I feel comfortable around people.,👆 Agree (as it reflects agreeing with the sent...,4,4,5.0
3,I keep in the background.,👁️‍🗨️ Neutral (This indicates that you neither...,2,4,NaN
4,I start conversations.,\nAGREE. You seem to be someone who initiates ...,4,3,5.0


In [127]:
# compute one number of how the percentage of correct answers
print(f"Exact Matches: {(df['CLONE_eli'] == df['IRL_eli']).sum() / len(df)}")
print(f"Correlation: {df['CLONE_eli'].corr(df['IRL_eli'])}")
df['elias_correct'] = df['CLONE_eli'] == df['IRL_eli']

Exact Matches: 0.14
Correlation: 0.09891782353111638


### Airi below

In [82]:
#### Load File
import pandas as pd
df = pd.read_csv(f'results/{save}_simulation.csv')
# df = df.drop(df.columns[0], axis=1) #if loaded from csv, drop the added index col
df.head()
#### Cleanup
# remove all characters from a black list from the column answer
# for substr in utils.BLACKLIST_ANSWER_SUBSTRINGS:
#      df['answer'] = df['answer'].apply(lambda x: re.sub(substr, "", x))
# # Update isValid
#      df['isValid'] = df['answer'].apply(lambda x: x in surv.POSSIBLE_ANSWERS)

# if all values in isValid is true, drop the column, else print a message
# if df['isValid'].all():
#     df = df.drop('isValid', axis=1)
# else:
#     print("Some answers were not valid")

df.head()
#### Proces simulation output
air = surv.test_answers["airidas"]
eli = surv.test_answers["elias"]
df.insert(2, "airidas", air[:len(df)])
df.insert(3, "elias", eli[:len(df)])

df['answer'] = df['answer'].str.upper()
df['airidas'] = df['airidas'].str.upper()
df['elias'] = df['elias'].str.upper()
remap_dict = {"I EXPECT IT": 5, "I LIKE IT": 4, "I AM NEUTRAL": 3, "I CAN TOLERATE IT": 2, "I DISLIKE IT": 1}
df['answer'] = df['answer'].map(remap_dict)
df['airidas'] = df['airidas'].map(remap_dict)
df['elias'] = df['elias'].map(remap_dict)
# compute one number of how the percentage of correct answers
print(f"Exact Matches: {(df['answer'] == df['airidas']).sum() / len(df)}")
print(f"Correlation: {df['answer'].corr(df['airidas'])}")

Exact Matches: 0.0
Correlation: nan
